In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

In [16]:
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
url = 'https://gearvn.com/collections/laptop'
driver.get(url)

wait = WebDriverWait(driver, 30)
# Sử dụng JavaScript để ẩn pop-up
driver.execute_script("document.getElementById('popup-promotion').style.display = 'none';")

def is_view_more_button_displayed():
    try:
        view_more_button = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, 'load_more'))
        )
        return view_more_button.is_displayed()
    except:
        return False

def get_product_count():
    # Sử dụng một selector chính xác hơn để đếm số lượng sản phẩm
    product_elements = driver.find_elements(By.CLASS_NAME, 'col-xl-3.col-lg-3.col-6.proloop.ajaxloop.loaded')
    return len(product_elements)
def is_product_count_changed(previous_product_count):
    current_product_count = get_product_count()
    return current_product_count != previous_product_count

# Vòng lặp để click vào nút "Xem thêm sản phẩm" cho đến khi nó không còn hiển thị
while is_view_more_button_displayed():
    initial_product_count = get_product_count()    
    view_more_button = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, 'load_more'))
    )
    driver.execute_script("arguments[0].scrollIntoView();", view_more_button)
    
    # Đợi cho dữ liệu mới tải lên
    try:
        WebDriverWait(driver, 30).until(lambda driver: is_product_count_changed(initial_product_count))
    except TimeoutException:
        pass
    # Thực hiện click lại nếu có lỗi ElementClickInterceptedException
    try:
        view_more_button.click()
    except ElementClickInterceptedException:
        pass
        view_more_button.click()

    # Đợi một khoảng thời gian trước khi kiểm tra số lượng sản phẩm, nếu không có sự thay đổi thì break (Tức là đã hiển thị toàn bộ sản phẩm)
    sleep(3)
    if not is_product_count_changed(initial_product_count):
        break

# Đợi cho các liên kết sản phẩm trở nên hiển thị
product_links = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'proloop-name')))

# Trích xuất thuộc tính href từ các liên kết
all_product_links = [link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in product_links]

print(all_product_links)

driver.quit()

['https://gearvn.com/products/laptop-lenovo-ideapad-slim-3-15abr8-82xm00ehvn', 'https://gearvn.com/products/laptop-gaming-msi-stealth-18-ai-studio-a1vhg-021vn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-5-15iru9-83d0000evn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-5-15iru9-83d00003vn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-3-15abr8-82xm00ejvn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-5-14imh9-83da0020vn', 'https://gearvn.com/products/laptop-asus-vivobook-14-oled-a1405va-km257w', 'https://gearvn.com/products/laptop-asus-vivobook-15-oled-a1505va-l1491w', 'https://gearvn.com/products/laptop-gaming-lenovo-loq-15irx9-83dv00d5vn', 'https://gearvn.com/products/laptop-lenovo-thinkpad-e16-gen-1-21jn005mvn', 'https://gearvn.com/products/laptop-gaming-lenovo-loq-15irx9-83dv00ervn', 'https://gearvn.com/products/laptop-gaming-asus-rog-zephyrus-g16-gu605mv-qr196ws', 'https://gearvn.com/products/laptop-gaming-asus-tuf-gaming-a15-fa507uv-

In [17]:
print(len(all_product_links))

277


In [268]:
def get_product_info(all_product_links):
    driver.get(all_product_links)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    #PRODUCT_NAME
    try:
        product_name = soup.find('div', class_ = 'product-name').text.strip()
    except:
        product_name = ''
        #PRODUCT_URL
    try:
        product_url = soup.find('link', {'rel': 'canonical'}).get('href')
    except:
        product_url = ''
    try:
        discounted_price = soup.find('span', class_ = 'pro-price a').text.strip()
    except:
        discounted_price = ''
    try:
        original_price = soup.find('del').text.strip()
    except:
        original_price = discounted_price    
    try:   
        discount_percentage = soup.find('span', class_ = 'pro-percent').text.strip().replace('-','')
    except:
        discount_percentage = ''
    
    # Tạo một từ điển để lưu các giá trị
    promotions = []

    try:
        gift_promo_elements = soup.find_all('li', class_='higtlight')
        for gift_promo_element in gift_promo_elements:
        # Trích xuất thông tin từ mỗi phần tử và thêm vào từ điển
            promotion_text = gift_promo_element.text.strip()
            promotions.append(promotion_text )
    except AttributeError:
        pass

    try:
        promotion_section = soup.find('div', class_='product-desc-short prtab2')
        # Kiểm tra xem phần tử chứa thông tin quà tặng có tồn tại không
        if promotion_section.find('strong', string='Quà tặng:'):
            gift_section = promotion_section.find('strong', string='Quà tặng:')
            if gift_section.find_next('span').find_all('br'):
                gift_section_2 = gift_section.find_next('span').find_all('br')
                for br_tag in gift_section_2:
                    if br_tag.previous_sibling:
                        promotions.append(br_tag.previous_sibling.strip().replace('\xa0',' ').replace('🎁 ','').replace('🎁',''))
                    if br_tag.find_next_sibling:
                        promotions.append(br_tag.next_sibling.strip().replace('\xa0',' ').replace('🎁 ','').replace('🎁',''))
            else:
                gift_find = gift_section.find_parent('p')
                siblings = gift_find.find_next_sibling()

                for sibling in siblings:
                    if sibling.find_all('br'):
                        for br_tag in sibling.find_all('br'):
                            if br_tag.previous_sibling:
                                promotions.append(br_tag.previous_sibling.strip().replace('\xa0',' ').replace('🎁 ','').replace('🎁',''))
                            if br_tag.next_sibling:
                                promotions.append(br_tag.next_sibling.strip().replace('\xa0',' ').replace('🎁 ','').replace('🎁',''))
                    else:
                        promotion_more = sibling.text.strip().replace('\xa0',' ').replace('🎁 ','').replace('🎁','')
                        if promotion_more == '':
                            break
                        else:
                            promotions.append(promotion_more)
        
        if promotion_section.find_next('p'):
            if promotion_section.find_next('p').find_all('br'):
                gift_box = promotion_section.find_next('p').find_all('br')
                for gift in gift_box:
                    gift_content = gift.next_sibling.text.strip().replace('🎁 ','').replace('🎁','').replace('\xa0',' ')
                    promotions.append(gift_content)
            elif promotion_section.find_next('p'):
                if '🎁' in promotion_section.find_next('p').text.strip():
                    gift_text = promotion_section.find_next('p').text.strip().replace('\xa0',' ').replace('🎁 ','').replace('🎁','')
                    promotions.append(gift_text)
            else:
                pass
    except AttributeError:
        pass
    max_promotion = 3
    product_price = {
            'product_name': product_name,
            'discounted_price': discounted_price,
            'original_price': original_price,
            'discount_percentage': discount_percentage,
            'shop_brand': 'GearVN'
        }
    # Thêm các biến khuyến mại vào dictionary
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotions):
            value = promotions[i-1]
        else:
            value = ''

        product_price[key] = value

    product_price['product_url'] = product_url

    return product_price

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
for product_link in all_product_links:
    product_price = get_product_info(product_link)
    data.append(product_price)
    print(product_price)

driver.quit()

{'product_name': 'Laptop Lenovo Ideapad Slim 3 15ABR8 82XM00EHVN', 'discounted_price': '14.990.000₫', 'original_price': '15.990.000₫', 'discount_percentage': '6%', 'shop_brand': 'GearVN', 'promotion_1': 'Balo Lenovo Office', 'promotion_2': '', 'promotion_3': '', 'product_url': 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-3-15abr8-82xm00ehvn'}
{'product_name': 'Laptop gaming MSI Stealth 18 AI Studio A1VHG 021VN', 'discounted_price': '93.990.000₫', 'original_price': '94.990.000₫', 'discount_percentage': '1%', 'shop_brand': 'GearVN', 'promotion_1': 'Stealth Agent Backpack ( kèm máy)', 'promotion_2': 'Chuột MSI M99 Pro ( kèm máy)', 'promotion_3': '', 'product_url': 'https://gearvn.com/products/laptop-gaming-msi-stealth-18-ai-studio-a1vhg-021vn'}
{'product_name': 'Laptop Lenovo Ideapad Slim 5 15IRU9 83D0000EVN', 'discounted_price': '19.490.000₫', 'original_price': '20.990.000₫', 'discount_percentage': '7%', 'shop_brand': 'GearVN', 'promotion_1': 'Balo Lenovo Office', 'promotion_2

In [269]:
import pandas as pd
df = pd.DataFrame(data)
df

,product_name,discounted_price,original_price,discount_percentage,shop_brand,promotion_1,promotion_2,promotion_3,product_url
0,Laptop Lenovo Ideapad Slim 3 15ABR8 82XM00EHVN,14.990.000₫,15.990.000₫,6%,GearVN,Balo Lenovo Office,,,https://gearvn.com/products/laptop-lenovo-idea...
1,Laptop gaming MSI Stealth 18 AI Studio A1VHG 0...,93.990.000₫,94.990.000₫,1%,GearVN,Stealth Agent Backpack ( kèm máy),Chuột MSI M99 Pro ( kèm máy),,https://gearvn.com/products/laptop-gaming-msi-...
2,Laptop Lenovo Ideapad Slim 5 15IRU9 83D0000EVN,19.490.000₫,20.990.000₫,7%,GearVN,Balo Lenovo Office,,,https://gearvn.com/products/laptop-lenovo-idea...
3,Laptop Lenovo Ideapad Slim 5 15IRU9 83D00003VN,20.490.000₫,21.990.000₫,7%,GearVN,Balo Lenovo Office,,,https://gearvn.com/products/laptop-lenovo-idea...
4,Laptop Lenovo Ideapad Slim 3 15ABR8 82XM00EJVN,13.490.000₫,13.990.000₫,4%,GearVN,Balo Lenovo Office,,,https://gearvn.com/products/laptop-lenovo-idea...
...,...,...,...,...,...,...,...,...,...
272,Laptop Acer Aspire 7 A715 76 57CY,14.490.000₫,16.490.000₫,12%,GearVN,,,,https://gearvn.com/products/laptop-acer-aspire...
273,Laptop gaming ASUS ROG Strix SCAR 16 G634JZR N...,101.990.000₫,110.000.000₫,7%,GearVN,Balo ASUS ROG Backpack (kèm máy),,,https://gearvn.com/products/laptop-gaming-asus...
274,Laptop Acer Swift X SFX14 71G 75CV,39.990.000₫,40.990.000₫,2%,GearVN,Giảm thêm 500.000đ cho HSSV (đến 30.09.2023),,,https://gearvn.com/products/laptop-acer-swift-...
275,Laptop gaming Lenovo Yoga Pro 7 14IRH8 82Y70050VN,36.990.000₫,94.990.000₫,,GearVN,,,,https://gearvn.com/products/laptop-gaming-leno...


In [271]:
df.to_excel('gearvn_price.xlsx', index=False)